In [62]:
import requests,re,os,time
import datetime
from bs4 import BeautifulSoup,NavigableString

#抓取時間 20060101~20160531
forum_list = ['Gossiping','Hate','WomenTalk','AllTogether'] # 14393 41-3375 70-5016
    



class PTT_crawler(object):
    def __init__(self, forum, until, sleep=True):
        self.forumlist = ['timecapsule','relationship','girl','makeup','dressup','funny','rainbow','marvel','boy','horoscopes','food','mood','travel']
        self.forum = forum
        self.until = until if until <= str(datetime.datetime.now())[0:10] else str(datetime.datetime.now())[0:10]
        self.sleep = sleep
        self.post_ids = list()
        self.url_num = None
        self.data = dict()
        print("PTT_crawler is activated...")

    def crawler(self):
        rs = requests.session()
        payload = {'from':'/bbs/%s/index.html' % self.forum,'yes':'yes'}
        res = rs.post('https://www.ptt.cc/ask/over18', data=payload)
        assert rs.get("https://www.ptt.cc/bbs/{}/index.html".format(self.forum)).status_code==200,"the forum you type doesn't exist"
        page_num = 1
        while page_num <= 100: #指定抓取的頁碼
            page_url = 'https://www.ptt.cc/bbs/%s/index%s.html' % (self.forum, str(page_num))
            soup = BeautifulSoup(rs.get(page_url).text,'lxml')
            self.post_ids.append(list(map(lambda x:"https://www.ptt.cc/"+x.find('a')['href'],soup.select('.r-ent'))))
            page_num += 1
        self.post_ids = sum(self.post_ids,[])
        self.url_num = len(self.post_ids)
        print(self.url_num,"post urls are collected")        
        self.data['info'] = {'until':self.until}
        self.data['post'] = [None]*self.url_num
        for idx,pid in enumerate(self.post_ids):
#             print(pid)
            res = rs.get(pid, verify=False)
            soup = BeautifulSoup(res.text,'lxml')
            if soup.select('.article-meta-tag'):
                contents = soup.select('#main-content')[0].contents
                all_text = ""
                end_row = argmax(list(map(lambda x:x.name=='span' and x.attrs['class'][0]=='f2' and '發信站' in x.text,contents)))
                for num in range(4,end_row):
                    if isinstance(contents[num],NavigableString):
                        tmp = re.sub('\s','',contents[num])
                        if len(tmp) != 0:
                            all_text += re.sub('\s','',contents[num])+'\n'
                    elif contents[num].name == 'a':
                        all_text += contents[num].text+'\n'
                all_text = re.sub(r'\-\-.+\d$','',all_text)
                self.data['post'][idx] = {'content':all_text}
                info = soup.select('#main-content')[0].select('.article-meta-value') # 抓取 meta-tag:author,forum,title,time
    #             if len(info) == 4:
                self.data['post'][idx]['metatag'] = list(map(lambda x: x.text,soup.select('#main-content')[0].select('.article-meta-value')))
    #             except:
    #                 pass
    #             page_num += 1
    #             if page_num % 500 == 0:
    #                 time.sleep(5)

In [65]:
for i in range(100):
    print(c.data['post'][i])

{'content': '1Gossiping綜合◎記者，你的名字是抄襲。HOTlinkinpork/to2ˇHate心情◎幹!!期末考要加油啊HOTloverlover/ci3ˇFORMULA1賽車◎SadGRANDPRIXHOTMSCHUMACHER/S4ˇsex男女◎★原來現在政府鼓勵中出★HOTtoshizou/badb5ˇBeauty聊天◎★Beauty板禮貌運動！★HOTno1great/high6ˇCMWang*MLB◎就這樣~~小熊被老虎吃了HOTNYYAndy/geniu7ˇWarCraft即時●★那你剛剛為什麼不買purge刀★96Trueapril/sho8ˇjoke娛樂◎今天唯一的高潮77opengoodbook/9ˇStupidClown個人◎"我二度灼傷""..攝氏二度嗎?"56antipower/big10ˇshare娛樂●雪兒～快來吃歐啪麵包吧54stkeiko/KaBik雖然是冷門時間不過第一就是第一--不行po的話快回信給我我自d--\n', 'metatag': ['gogoegg (--)', 'Gossiping', '第一耶！', 'Mon Jun 20 07:11:31 2005']}
{'content': '', 'metatag': ['Muroi (I Honestly Love You)', 'Gossiping', 'Re: (問題)華航空難留言', 'Mon Jun 20 07:28:27 2005']}
{'content': '那不是為了反盜版那是為了捐錢給消基會當年美國唱了WEARETHEWORLD之後台灣這些歌手也覺得該如法泡製一番但當年國家安定人民富足實在找不出啥理由於是就拿來捐給消基會了至於為後來很多人會誤傳是為了反盜版我也不知道大慨是捐給消基會這個理由蠻妙的吧!以上是我記憶的東西網路上找到一篇文可以佐證\nhttp://www.zjcx.gov.cn/portal/main/DJLZG_59/article2137.html\n當年大合唱的歌曲還有滾石的快樂天堂唱給動物園搬家用的還有飛碟的飛向未來好像只是幾周年慶用的至於後來八九天安門屠殺後的歷史的傷口讓童安格被大陸禁了好幾年\n--\n', 'metatag': ['JCC (JCC                    )', 'Gossi

In [63]:
c=PTT_crawler('Gossiping','1')
c.crawler()

PTT_crawler is activated...
2000 post urls are collected


In [53]:
import requests,re,os,time
from numpy import argmax
from bs4 import BeautifulSoup,NavigableString
payload = {
'from':'/bbs/%s/index.html' % 'Gossiping',
'yes':'yes'
}
rs = requests.session()
res = rs.post('https://www.ptt.cc/ask/over18', data=payload)
page_num = 1
page_url = 'https://www.ptt.cc//bbs/Gossiping/M.1119237467.A.182.html'
res = rs.get(page_url)
# soup = BeautifulSoup(res.text,'lxml')
# res = rs.get("https://www.ptt.cc/"+soup.select('.r-ent')[0].find('a')['href'])
soup = BeautifulSoup(res.text,'lxml')
contents = soup.select('#main-content')[0].contents

for idx,i in enumerate(contents):
    print(idx,type(i),i)

all_text = ""
end_row = argmax(list(map(lambda x:x.name=='span' and x.attrs['class'][0]=='f2' and '發信站' in x.text,contents)))
print(end_row)
for num in range(4,end_row):
#     print('發信站' in contents[num])
    if isinstance(contents[num],NavigableString):
        tmp = re.sub('\s','',contents[num])
        if len(tmp) != 0:
            all_text += re.sub('\s','',contents[num])+'\n'
    elif contents[num].name == 'a':
        all_text += contents[num].text+'\n'

print(all_text)
# print(contents[79].name)
# print(contents[79].attrs)
# contents[0:3]
# argmax(list(map(lambda x:x.name=='span' and x.attrs['class'][0]=='f2',contents)))

0 <class 'bs4.element.Tag'> <div class="article-metaline"><span class="article-meta-tag">作者</span><span class="article-meta-value">sallycat (那就這樣吧...)</span></div>
1 <class 'bs4.element.Tag'> <div class="article-metaline-right"><span class="article-meta-tag">看板</span><span class="article-meta-value">Gossiping</span></div>
2 <class 'bs4.element.Tag'> <div class="article-metaline"><span class="article-meta-tag">標題</span><span class="article-meta-value">Re: 請問有無胡呱女兒的男友八卦?!</span></div>
3 <class 'bs4.element.Tag'> <div class="article-metaline"><span class="article-meta-tag">時間</span><span class="article-meta-value">Mon Jun 20 11:23:10 2005</span></div>
4 <class 'bs4.element.NavigableString'> 

5 <class 'bs4.element.Tag'> <span class="f2">※ 引述《psgirl (  [噗啾～！]     )》之銘言：
</span>
6 <class 'bs4.element.Tag'> <span class="f6">:     胡X女兒的男朋友他媽媽是我弟他們班導師(阿阿 關係好複雜＝　＝；)
</span>
7 <class 'bs4.element.Tag'> <span class="f6">: 　　胡Ｘ禎他們之前有養拉布拉多
</span>
8 <class 'bs4.element.Tag'> <span class="f6">: 　　但是

In [7]:
None=='a'

False

In [8]:
c.data['post'][0]

{'content': '1Gossiping綜合◎記者，你的名字是抄襲。HOTlinkinpork/to2ˇHate心情◎幹!!期末考要加油啊HOTloverlover/ci3ˇFORMULA1賽車◎SadGRANDPRIXHOTMSCHUMACHER/S4ˇsex男女◎★原來現在政府鼓勵中出★HOTtoshizou/badb5ˇBeauty聊天◎★Beauty板禮貌運動！★HOTno1great/high6ˇCMWang*MLB◎就這樣~~小熊被老虎吃了HOTNYYAndy/geniu7ˇWarCraft即時●★那你剛剛為什麼不買purge刀★96Trueapril/sho8ˇjoke娛樂◎今天唯一的高潮77opengoodbook/9ˇStupidClown個人◎"我二度灼傷""..攝氏二度嗎?"56antipower/big10ˇshare娛樂●雪兒～快來吃歐啪麵包吧54stkeiko/KaBik雖然是冷門時間不過第一就是第一--不行po的話快回信給我我自d--',
 'metatag': [<span class="article-meta-value">gogoegg (--)</span>,
  <span class="article-meta-value">Gossiping</span>,
  <span class="article-meta-value">第一耶！</span>,
  <span class="article-meta-value">Mon Jun 20 07:11:31 2005</span>]}